In [26]:
import multiprocessing
import os
import time
import numpy as np
from scipy.spatial import cKDTree
import open3d as o3d
import util
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
import random
from BendLength import BendLengthCalculator

In [27]:
point_cloud_location = "/home/chris/Code/PointClouds/sampled_model.ply"
pcd = o3d.io.read_point_cloud(point_cloud_location)

In [28]:
front =  [-0.47452876114542436, 0.57451207113849134, -0.66690204300328082]
lookat = [-6.3976792217838847, 20.927374714553928, 18.659758576873813]
up =  [-0.056918726368614558, -0.77607794684805009, -0.62806311705487861]
zoom = 0.69999999999999996

In [53]:
def multiOrderRansac(pcd, pt_to_plane_dist, visualize=False, verbose=False):

    if verbose:
        print('Using planar patches to detect number of planes')
    oboxes = pcd.detect_planar_patches(
    normal_variance_threshold_deg=20,
    coplanarity_deg=75,
    outlier_ratio=0.2,
    min_plane_edge_length=0,
    min_num_points=0,
    search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30)
)

    max_plane_idx = len(oboxes)  # Use this as max_plane_idx
    if verbose:
        print(f'Found {max_plane_idx} planes')
    segment_models = {}
    segments = {}
    segment_indices = {}
    main_surface_idx = 0
    largest_surface_points = 0
    rest = pcd
    rest_indices = np.arange(len(pcd.points))
    
    if verbose:
        print('Running multi-order RANSAC')

    for i in range(max_plane_idx):
        #print(f'Run {i}/{max_plane_idx} started. ', end='')
        colors = plt.get_cmap("tab20")(i)
        segment_models[i], inliers = rest.segment_plane(distance_threshold=pt_to_plane_dist,ransac_n=3,num_iterations=50000)
        segments[i] = rest.select_by_index(inliers)
        global_inliers = rest_indices[inliers]
        segment_indices[i] = global_inliers
        if len(segments[i].points) > largest_surface_points:
            largest_surface_points = len(segments[i].points) 
            main_surface_idx = i
        segments[i].paint_uniform_color(list(colors[:3]))
        rest = rest.select_by_index(inliers, invert=True)
        rest_indices = np.delete(rest_indices, inliers)
        #print('Done')

    if visualize:
        o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)],zoom=zoom,front=front,lookat=lookat,up=up)

    print(f'Largest surface is {largest_surface_points} points, index {main_surface_idx}')
    
    return segment_models, segments, segment_indices, main_surface_idx

In [64]:
# Preprocess the point cloud
pcd = util.preProcessCloud(pcd)
pcd_points = np.asarray(pcd.points)

# Detect planes, intersections, and anchor points
segment_models, segments, segment_indices, main_surface_idx = multiOrderRansac(pcd, pt_to_plane_dist=0.001, verbose=True, visualize=True)
print(f'segment_models: {segment_models}')

Using planar patches to detect number of planes
Found 7 planes
Running multi-order RANSAC
Largest surface is 821277 points, index 0
segment_models: {0: array([-7.29786835e-10, -2.47882753e-09,  1.00000000e+00, -1.00000001e+00]), 1: array([-5.73576663e-01,  2.51548819e-08,  8.19151886e-01,  8.57598890e+01]), 2: array([ 5.73576566e-01, -3.50669055e-08,  8.19151954e-01, -8.63130885e+01]), 3: array([   0.28678808,    0.4967315 ,    0.81915226, -117.80455137]), 4: array([ -0.28678815,   0.49673157,   0.81915219, -31.76811987]), 5: array([ 0.28678812, -0.49673157,  0.8191522 , 31.21490729]), 6: array([ -0.28678802,  -0.4967315 ,   0.81915228, 117.25131757])}


In [65]:
def findAnglesBetweenPlanes(segment_models, main_surface_idx):
    angles_rad = {}
    angles_deg = {}

    for i in range(len(segment_models)):
        if i != main_surface_idx:
            dot_product = np.dot(segment_models[main_surface_idx][:3], segment_models[i][:3])
            angles_rad[i] = np.arccos(np.clip(dot_product, -1, 1))
            angles_deg[i] = angles_rad[i] * 180 / np.pi
            print(angles_deg[i])

    return angles_rad

In [66]:
angles_rad = findAnglesBetweenPlanes(segment_models, main_surface_idx)


35.00001578462026
35.00000910471435
34.99997909410215
34.99998583979463
34.99998451962698
34.99997653411315


In [67]:
def findIntersectionLines(segment_models, main_surface_idx):

    intersection_lines = {}

    for i in range(len(segment_models)):
        if i != main_surface_idx:
            cross_product = np.cross(segment_models[main_surface_idx][:3], segment_models[i][:3])
            if abs(abs(cross_product[2]) < 0.0001):
                common_point = np.linalg.solve([segment_models[main_surface_idx][1:3], segment_models[i][1:3]],
                                        [-segment_models[main_surface_idx][-1], -segment_models[i][-1]])
            else:
                common_point = np.linalg.solve([segment_models[main_surface_idx][:2], segment_models[i][:2]],
                                        [-segment_models[main_surface_idx][-1], -segment_models[i][-1]])

            common_point = np.asarray([common_point[0], common_point[1], 0])
                
            intersection_lines[i] = [cross_product,common_point]

    return intersection_lines

In [69]:
def findIntersectionLines(segment_models, main_surface_idx):
    """
    Computes intersection lines between the main plane and other segment planes.
    Each line is represented by (direction_vector, point_on_line).
    """
    intersection_lines = {}

    main_plane = segment_models[main_surface_idx]
    n1 = main_plane[:3]
    d1 = -main_plane[3]

    for i, plane in segment_models.items():
        if i == main_surface_idx:
            continue

        n2 = plane[:3]
        d2 = -plane[3]

        # Direction of intersection = cross product of normals
        direction = np.cross(n1, n2)
        if np.linalg.norm(direction) < 1e-6:
            continue  # Planes are parallel or identical

        # Build coefficient matrix and solve for point on line
        A = np.array([n1, n2, direction])
        b = np.array([d1, d2, 0])

        # Use least-squares to solve A x = b (more stable than direct solve)
        try:
            point_on_line = np.linalg.lstsq(A, b, rcond=None)[0]
        except np.linalg.LinAlgError:
            continue  # Skip if system is badly conditioned

        intersection_lines[i] = [direction, point_on_line]

    return intersection_lines


In [70]:
intersection_lines = findIntersectionLines(segment_models, main_surface_idx)
print(intersection_lines)

{1: [array([-2.71854181e-08, -5.73576662e-01, -1.42179764e-09]), array([ 1.50945892e+02, -7.15675734e-06,  1.00000012e+00])], 2: [array([3.30363691e-08, 5.73576567e-01, 1.42179741e-09]), array([ 1.49054096e+02, -8.58756827e-06,  1.00000012e+00])], 3: [array([-4.96731501e-01,  2.86788078e-01,  3.48390073e-10]), array([101.97896599, 176.6327429 ,   1.00000052])], 4: [array([-4.96731568e-01, -2.86788154e-01, -1.07340653e-09]), array([-26.97895227,  46.72890801,   1.0000001 ])], 5: [array([4.96731567e-01, 2.86788119e-01, 1.07340644e-09]), array([-27.92485106,  48.36725829,   1.00000011])], 6: [array([ 4.96731494e-01, -2.86788017e-01, -3.48389927e-10]), array([102.9248393 , 178.27107889,   1.00000052])]}


In [75]:
def findAnchorPoints(segment_models, segments, intersection_lines, main_surface_idx, visualize=False):
    anchor_points = {}
    max_plane_idx = len(segment_models)
    for i in range(len(segment_models)):
        if i != main_surface_idx:
            segment_center = segments[i].get_center()
            b_vec = [sc - il for sc, il in zip(segment_center, intersection_lines[i][1])]
            a_vec = intersection_lines[i][0]/np.linalg.norm(intersection_lines[i][0])
            p_vec = np.dot(a_vec,b_vec)/np.dot(a_vec,a_vec) * a_vec
            anchor_points[i] = intersection_lines[i][1] + p_vec
    
    if visualize:
        sphere_radius = 2  # Adjust the radius based on your scale
        spheres = []
        for point in anchor_points.values():
            sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
            sphere.translate(point)  # Move the sphere to the anchor point
            sphere.paint_uniform_color([1, 0, 0])  # Red color for visibility
            spheres.append(sphere)

        # Visualize
        o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+spheres, zoom=zoom, front=front, lookat=lookat, up=up)
    
    return anchor_points

In [76]:
anchor_points = findAnchorPoints(segment_models, segments, intersection_lines, main_surface_idx, visualize=True)

In [77]:
sample_dist = 0.3
aggregation_range = 15
eigen_threshold = 0.05
angle_threshold = 0.12
radius = 1.5
bend_length_calculator = BendLengthCalculator(pcd, anchor_points, intersection_lines, eigen_threshold, angle_threshold, aggregation_range, sample_dist, radius)
bend_edges = bend_length_calculator.compute_bend_lengths()